# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

In [8]:
%%capture
!pip install transformers

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [9]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader

In [10]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"


encoded_input = tokenizer(prompt, return_tensors='pt')
print(f"inputs_ids shape:{encoded_input['input_ids'].shape}\nattention_mask shape:{encoded_input['attention_mask'].shape}\n")
print(encoded_input)
token = tokenizer.decode(encoded_input['input_ids'][0])
print(f"tokens: {token}\n")

with torch.no_grad():
    output = model(**encoded_input)


logits = output.logits
print(logits.shape) #shape: [B, L, V]

last_token_logits = logits[0, -1, :]  #これで一番最後のトークンの次のロジット
print(last_token_logits.shape)

last_token_logits_prob = F.softmax(last_token_logits, dim=0)

topk = torch.topk(last_token_logits_prob, k=10)

for idx, score in zip(topk.indices, topk.values):
    print(f"{tokenizer.decode(idx)}: {score.item():.4f}")

inputs_ids shape:torch.Size([1, 5])
attention_mask shape:torch.Size([1, 5])

{'input_ids': tensor([[ 464, 3807,  373, 1336,  286]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tokens: The movie was full of

torch.Size([1, 5, 50257])
torch.Size([50257])
 jokes: 0.0219
 great: 0.0186
 laughs: 0.0115
 bad: 0.0109
 surprises: 0.0107
 references: 0.0105
 fun: 0.0100
 humor: 0.0074
 ": 0.0074
 the: 0.0067


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [11]:
#greedy
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=20)
  end_time = time.time()

print(f"time: {(end_time - start_time):.4f}")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #確率が最も高いトークンを１つずつ順番に選んでいく->greedy decoding デフォはこれっぽい


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 1.4643
torch.Size([1, 20])
The movie was full of jokes and jokes about how the movie was a joke. It was a joke


In [12]:
#beam_serch
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=15, num_beams=5)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #ビームサーチは貪欲法と比較して処理時間がおそい．


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 3.437s
torch.Size([1, 15])
The movie was full of jokes and jokes and jokes and jokes and jokes and


In [13]:
#topk
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(
      encoded_input['input_ids'],
      max_length=15,
      top_k=50,
      top_p=1.0,
      temperature=1.0)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 1.977s
torch.Size([1, 15])
The movie was full of jokes and jokes about how the movie was a joke


## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [14]:
import time

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "The movie was full of"
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']


with torch.no_grad():
  start_time = time.time()

  output = model.generate(
      input_ids,
      max_length=15,
      top_k=50,
      top_p=1.0,
      temperature=1.0,
      return_dict_in_generate=True,
      output_scores=True
  )
  end_time = time.time()



generated_ids = output.sequences[0]
print(f"generated_ids shape: {generated_ids.shape}")
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print(f"time: {(end_time - start_time):.3f}s")
print(f"\nGenerated Text:\n{generated_text}\n")

print(len(output.scores)) #tuple: max_len - input_len (=generated_len)
print(output.scores[0].shape) #shape: [1, vocab_size]

for score in output.scores:
  prob = F.softmax(score, dim=1)
  vocab_ids = torch.argmax(score, dim=1) #shape: [batch_size, ]
  tokens = tokenizer.decode(vocab_ids)
  print(f"{tokens}: P={prob[0, vocab_ids].item():.3f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated_ids shape: torch.Size([15])
time: 1.105s

Generated Text:
The movie was full of jokes and jokes about how the movie was a joke

10
torch.Size([1, 50257])
 jokes: P=0.022
 and: P=0.289
 jokes: P=0.099
 about: P=0.206
 how: P=0.100
 the: P=0.085
 movie: P=0.036
 was: P=0.296
 a: P=0.068
 joke: P=0.174


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [32]:
import math

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

sentences = ['The movie was full of surprises',
             'The movies were full of surprises',
             'The movie were full of surprises',
             'The movies was full of surprises']

def cal_ppl(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids']) #loss = -1/L Sigma log(wt|wt<)
        loss = outputs.loss
        ppl = math.exp(loss.item()) #tensorだったからitem()でfloat型に変換
    return ppl


for sentence in sentences:
  ppl = cal_ppl(sentence)
  print(ppl)


99.35393766125713
126.48173421763245
278.87789150142044
274.66100919346405


## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [70]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "What do you call a sweet eaten after dinner? It is called"
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=10,
        do_sample=True,
        top_p=0.90,
        top_k=50,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== 応答 ===")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== 応答 ===
What do you call a sweet eaten after dinner? It is called a sweet roasted meat.

Asteroids


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
